In [1]:
import random
import math
import logging
import time
import tensorflow as tf
from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

# Lire le fichier CSV principal
df = pd.read_csv('../../full_dataset.csv')
df = df.drop("date", axis=1)

# Identifier les coordonnées uniques (latitude, longitude)
coordinates = df[['latitude', 'longitude']].drop_duplicates()

# Liste pour stocker les dataframes échantillonnés
sampled_dfs = []

# Filtrer les données pour chaque bâtiment en fonction des coordonnées
for index, row in coordinates.iterrows():
    lat, long = row['latitude'], row['longitude']
    building_data = df[(df['latitude'] == lat) & (df['longitude'] == long)]
    
    # Calculer le nombre de lignes correspondant à 20% du dataframe
    num_rows = int(len(building_data) * 0.2)
    
    # Prendre les premières 20% des lignes
    sampled_df = building_data.head(num_rows)
    
    # Ajouter le dataframe échantillonné à la liste
    sampled_dfs.append(sampled_df)

# Fusionner tous les dataframes échantillonnés en un seul dataframe
data = pd.concat(sampled_dfs, ignore_index=True)

# save the dataframe to a csv file
data.to_csv('data.csv', index=False)

# Afficher un aperçu du dataframe final
print(data.head())
data_train = data[data['building_id'] != 8]
data_test = data[data['building_id'] == 8]

data_train.shape, data_test.shape


target_column = 'production'

x_train = data_train.drop(target_column, axis=1)
y_train = data_train[target_column].values.reshape(-1, 1)

x_test = data_test.drop(target_column, axis=1)
y_test = data_test[target_column].values.reshape(-1, 1)


x_scaler = MinMaxScaler(feature_range=(0, 1))
x_scaler.fit(x_train)

x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)


def get_windows(x, y, window_size):
    x_windows, y_windows = [], []

    for i in range(len(x) - window_size):
        x_window = x[i:i+window_size]
        y_window = y[i:i+window_size]

        x_window = np.hstack((x_window, y_window))

        x_windows.append(x_window)
        y_windows.append(y[i+window_size])

    return np.array(x_windows), np.array(y_windows)


x_train_windows, y_train_windows = get_windows(x_train_scaled, y_train, 10)
x_test_windows, y_test_windows = get_windows(x_test_scaled, y_test, 10)


tf.random.set_seed(42)


def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print(f"MSE: {mse}")
    return mse


# Define the parameter ranges
LSTM1_units = list(range(16, 256, 8))
LSTM1_activation = ['tanh', 'relu', 'sigmoid', 'hard_sigmoid', 'linear']
DROPOUT1_rate = list(np.arange(0, 0.9, 0.01))
LSTM2_units = list(range(16, 256, 8))
LSTM2_activation = ['tanh', 'relu', 'sigmoid', 'hard_sigmoid', 'linear']
DROPOUT2_rate = list(np.arange(0, 0.9, 0.01))
DENSE1_units = list(range(16, 256, 8))
DENSE1_activation = ['tanh', 'relu', 'sigmoid', 'hard_sigmoid', 'linear']
DENSE2_units = [1]
DENSE2_activation = ['linear']
OPTIMIZER_learning_rate = list(np.arange(0, 0.001, 0.0001))
EPOCHS = list(range(200, 400, 10))
BATCH_SIZE = list(range(32, 128, 8))


def objective_function(params):
    # Unpack the parameters with names
    lstm1_units, lstm1_activation, dropout1_rate, lstm2_units, lstm2_activation, dropout2_rate, dense1_units, dense1_activation, dense2_units, dense2_activation, optimizer_learning_rate, epochs, batch_size = params
    print(f"lstm1_units: {lstm1_units}, activation: {lstm1_activation}, dropout1_rate: {dropout1_rate}")
    print(f"lstm2_units: {lstm2_units}, activation: {lstm2_activation}, dropout2_rate: {dropout2_rate}")
    print(f"dense1_units: {dense1_units}, activation: {dense1_activation}")
    print(f"dense2_units: {dense2_units}, activation: {dense2_activation}")
    print(f"optimizer_learning_rate: {optimizer_learning_rate}, epochs: {epochs}, batch_size: {batch_size}")

    model = Sequential([
        LSTM(lstm1_units, activation=lstm1_activation, input_shape=(
            x_train_windows.shape[1:]), return_sequences=True),
        Dropout(dropout1_rate),

        LSTM(lstm2_units, activation=lstm2_activation, return_sequences=False),
        Dropout(dropout2_rate),

        Dense(dense1_units, activation=dense1_activation),
        Dense(dense2_units, activation=dense2_activation)
    ])

    optimizer = Adam(learning_rate=optimizer_learning_rate)
    model.compile(optimizer=optimizer, loss='mean_absolute_error')

    try:

        start_time = time.time()
        history = model.fit(x=x_train_windows,
                            y=y_train_windows,
                            epochs=epochs,
                            batch_size=batch_size,
                            validation_split=0.2,
                            shuffle=False,
                            verbose=0)
        training_duration = time.time() - start_time
    finally:
        print("finito pipo")
        # tracker.stop()

    y_pred_train = model.predict(x_train_windows)
    y_pred_test = model.predict(x_test_windows)
    performance_metric = evaluate_model(y_test_windows, y_pred_test)

    return performance_metric


def simulated_annealing(initial_temp, final_temp, alpha, max_iterations):
    # Initialize the default solution 
    current_solution = [
        128, "tanh", 0.2,
        64, "tanh", 0.2,
        64, "relu", 1, "linear",
        0.001, 300, 128
    ]
    current_mse = objective_function(current_solution)
    best_solution = current_solution
    best_mse = current_mse
    temp = initial_temp

    for iteration in range(max_iterations):
        if temp < final_temp:
            break

        # Generate a neighboring solution
        neighbor = list(current_solution)
        param_index = random.randint(0, len(current_solution) - 1)
        if param_index == 0:
            neighbor[param_index] = random.choice(LSTM1_units)
        elif param_index == 1:
            neighbor[param_index] = random.choice(LSTM1_activation)
        elif param_index == 2:
            neighbor[param_index] = random.choice(DROPOUT1_rate)
        elif param_index == 3:
            neighbor[param_index] = random.choice(LSTM2_units)
        elif param_index == 4:
            neighbor[param_index] = random.choice(LSTM2_activation)
        elif param_index == 5:
            neighbor[param_index] = random.choice(DROPOUT2_rate)
        elif param_index == 6:
            neighbor[param_index] = random.choice(DENSE1_units)
        elif param_index == 7:
            neighbor[param_index] = random.choice(DENSE1_activation)
        elif param_index == 8:
            neighbor[param_index] = random.choice(DENSE2_units)
        elif param_index == 9:
            neighbor[param_index] = random.choice(DENSE2_activation)
        elif param_index == 10:
            neighbor[param_index] = random.choice(OPTIMIZER_learning_rate)
        elif param_index == 11:
            neighbor[param_index] = random.choice(EPOCHS)
        elif param_index == 12:
            neighbor[param_index] = random.choice(BATCH_SIZE)

        neighbor_mse = objective_function(neighbor)

        # Acceptance criterion

        mse_diff = neighbor_mse - current_mse
        print(f"current_solution {current_solution}")
        print(f"neighbor {neighbor}")
        print(f"mse diff: {mse_diff}")
        if mse_diff < 0 or random.random() < math.exp(-mse_diff / temp):
            print("accept new solution", neighbor)
            current_solution = neighbor
            current_mse = neighbor_mse

            if neighbor_mse < best_mse:
                print("better solution")
                best_solution = neighbor
                best_mse = neighbor_mse

        # Update temperature
        temp = temp * alpha
        print(f"temp {temp}, mse {current_mse}, best_mse {best_mse}")

    return best_solution, best_mse


# Run the simulated annealing algorithm
initial_temp = 20  # Initial temperature
final_temp = 0.2  # Final temperature
alpha = 0.99  # Temperature decay rate
max_iterations = 10000  # Maximum number of iterations

best_params, best_mse = simulated_annealing(
    initial_temp, final_temp, alpha, max_iterations)
print("Best parameters:", best_params)
print("Best mse:", best_mse)

2024-05-20 15:10:40.747736: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 15:10:42.178627: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


   production   latitude  longitude   vmp  imp   voc   isc  p_per_m2  p_max  \
0         0.8  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
1        16.9  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
2         1.4  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
3         6.6  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
4         0.3  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   

   panel_area  ...  wind_speed_10m_std  wind_speed_10m_min  wind_speed_10m_q1  \
0        1.72  ...            0.903114                 1.3              1.900   
1        1.72  ...            0.969186                 0.9              2.975   
2        1.72  ...            0.306945                 0.4              1.100   
3        1.72  ...            0.491844                 0.4              0.800   
4        1.72  ...            0.527737                 1.6              2.275   

   wind_speed_10m_q2  wind_speed_10m_q

2024-05-20 15:10:45.214586: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-20 15:10:45.257762: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-20 15:10:45.257971: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

finito pipo
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MSE: 8.22560549636898
lstm1_units: 128, activation: tanh, dropout1_rate: 0.2
lstm2_units: 64, activation: tanh, dropout2_rate: 0.2
dense1_units: 64, activation: relu
dense2_units: 1, activation: linear
optimizer_learning_rate: 0.0004, epochs: 300, batch_size: 128


/home/quentin/Desktop/Optimization/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

finito pipo
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
MSE: 6.123188613503737
current_solution [104, 'linear', 0.26, 136, 'tanh', 0.2, 176, 'linear', 1, 'linear', 0.0005, 280, 128]
neighbor [104, 'linear', 0.26, 136, 'tanh', 0.2, 176, 'linear', 1, 'linear', 0.0005, 280, 128]
mse diff: -5.14776467201054
accept new solution [104, 'linear', 0.26, 136, 'tanh', 0.2, 176, 'linear', 1, 'linear', 0.0005, 280, 128]
temp 13.651091900207742, mse 6.123188613503737, best_mse 3.747674074094348
lstm1_units: 104, activation: linear, dropout1_rate: 0.26
lstm2_units: 136, activation: tanh, dropout2_rate: 0.2
dense1_units: 176, activation: tanh
dense2_units: 1, activation: linear
optimizer_learning_rate: 0.0005, epochs: 280, batch_size: 128


/home/quentin/Desktop/Optimization/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


finito pipo
60/60 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
MSE: 5.641257033107359
current_solution [104, 'linear', 0.26, 136, 'tanh', 0.2, 176, 'linear', 1, 'linear', 0.0005, 280, 128]
neighbor [104, 'linear', 0.26, 136, 'tanh', 0.2, 176, 'tanh', 1, 'linear', 0.0005, 280, 128]
mse diff: -0.4819315803963784
accept new solution [104, 'linear', 0.26, 136, 'tanh', 0.2, 176, 'tanh', 1, 'linear', 0.0005, 280, 128]
temp 13.514580981205665, mse 5.641257033107359, best_mse 3.747674074094348
lstm1_units: 32, activation: linear, dropout1_rate: 0.26
lstm2_units: 136, activation: tanh, dropout2_rate: 0.2
dense1_units: 176, activation: tanh
dense2_units: 1, activation: linear
optimizer_learning_rate: 0.0005, epochs: 280, batch_size: 128


/home/quentin/Desktop/Optimization/.venv/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


finito pipo
